## LangSmith를 활용한 LLM Evaluation

### 문제 상황
- LLM 챗봇을 만들었는데 **실제로 잘 작동하는지** 모르겠다
- 프롬프트를 수정했는데 **성능이 나아졌는지 확인**하고 싶다
- 사용자에게 배포하기 전에 **품질을 객관적으로 검증**하고 싶다

### 평가가 필요한 이유
1. **객관적 성능 측정**: "느낌적으로 좋다" → "80% 정확도" 수치화
2. **버전 비교**: 프롬프트 v1 vs v2 중 뭐가 더 나은지 비교
3. **약점 발견**: 어떤 유형의 질문에서 자주 틀리는지 파악
4. **지속적 개선**: 평가 → 개선 → 재평가 사이클 구축
5. **신뢰도 확보**: 특히 법률, 의료, 금융 같은 고위험 도메인

### LangSmith를 활용한 답변 평가
- LangSmith는 **LLM 애플리케이션의 평가 및 모니터링 도구**
- LangSmith의 역할
    - 평가 자동 반복 실행
    - 결과 저장 및 DB 관리
    - 웹 UI로 결과 시각화
    - 버전별 성능 비교
    - 팀 협업 기능

## LLM 답변 평가 전체 프로세스 
- 1. 평가 데이터셋 구축 (채점지) 
- 2. 평가 대상 (Target) 함수 정의
- 3. 평가자 (Evaluator) 정의
- 4. LangSmith로 평가 실행: evaluate() 
- 5. LangSmith 웹 UI에서 결과 확인

### 1. 평가 데이터셋 구축 (채점지) 
- Evaluation에 활용될 Question - Answer pair 생성

In [1]:
from langsmith import Client

client = Client()

# Define dataset: these are your test cases
dataset_name = "income_tax_dataset"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    inputs=[
        {"input_question": "제1조에 따른 소득세법의 목적은 무엇인가요?"},
        {"input_question": "'거주자'는 소득세법에서 어떻게 정의되나요?"},
        {"input_question": "'비거주자'는 소득세법에 따라 어떻게 정의되나요?"},
        {"input_question": "소득세법에 따른 '내국법인'은 누구를 의미하나요?"},
        {"input_question": "소득세법에 따라 소득세를 납부할 의무가 있는 사람은 누구인가요?"},
        {"input_question": "거주자의 과세 범위는 무엇인가요?"},
        {"input_question": "소득세법에 따라 소득은 어떻게 분류되나요?"},
        {"input_question": "종합소득이란 무엇인가요?"},
        {"input_question": "세금이 면제되는 소득의 종류는 무엇인가요?"},
        {"input_question": "소득세의 과세기간은 어떻게 되나요?"},
        {"input_question": "거주자의 소득세 납세지는 어디인가요?"},
        {"input_question": "비거주자의 소득세 납세지는 어디인가요?"},
        {"input_question": "납세지가 불분명한 경우 어떻게 되나요?"},
        {"input_question": "원천징수세액의 납세지는 어떻게 결정되나요?"},
        {"input_question": "납세자의 사망 시 납세지는 어떻게 되나요?"},
        {"input_question": "신탁 소득에 대한 납세의 범위는 무엇인가요?"},
        {"input_question": "원천징수 대상 소득은 무엇인가요?"},
        {"input_question": "공동 소유 자산의 양도소득은 어떻게 과세되나요?"},
        {"input_question": "이자 소득의 출처는 무엇인가요?"},
        {"input_question": "소득세법에서 배당소득은 어떻게 정의되나요?"}
    ],
    outputs=[
        {"output_answer": "소득세법의 목적은 소득의 성격과 납세자의 부담능력에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지하는 것입니다."},
        {"output_answer": "'거주자'는 한국에 주소를 두거나 183일 이상 거소를 둔 개인을 의미합니다."},
        {"output_answer": "'비거주자'는 거주자가 아닌 개인을 의미합니다."},
        {"output_answer": "'내국법인'은 법인세법 제2조 제1호에 따른 내국법인을 의미합니다."},
        {"output_answer": "거주자 및 국내원천소득이 있는 비거주자는 소득세를 납부할 의무가 있습니다."},
        {"output_answer": "거주자는 법에서 규정한 모든 소득에 대해 과세되며, 비거주자는 국내원천소득에 대해서만 과세됩니다."},
        {"output_answer": "소득은 종합소득, 퇴직소득, 양도소득으로 분류됩니다."},
        {"output_answer": "종합소득은 이자소득, 배당소득, 사업소득, 근로소득, 연금소득 및 기타소득을 포함합니다."},
        {"output_answer": "비과세 소득에는 공익신탁의 이익, 특정 사업소득 및 기타 법에서 정한 특정 소득이 포함됩니다."},
        {"output_answer": "소득세의 과세기간은 매년 1월 1일부터 12월 31일까지입니다."},
        {"output_answer": "거주자의 소득세 납세지는 주소지이며, 주소지가 없으면 거소지입니다."},
        {"output_answer": "비거주자의 소득세 납세지는 국내사업장의 소재지입니다. 국내사업장이 여러 곳인 경우 주된 사업장의 소재지가 납세지가 됩니다."},
        {"output_answer": "납세지가 불분명한 경우 대통령령으로 정합니다."},
        {"output_answer": "원천징수세액의 납세지는 원천징수자의 종류와 위치에 따라 결정됩니다."},
        {"output_answer": "납세자의 사망 시 상속인 또는 납세관리인의 주소지나 거소지가 납세지가 됩니다."},
        {"output_answer": "신탁 소득에 대한 납세의 범위는 신탁의 수익자가 해당 소득에 대해 납세의무를 집니다."},
        {"output_answer": "이자소득, 배당소득 및 기타 법에서 정한 소득은 원천징수 대상입니다."},
        {"output_answer": "공동 소유 자산의 양도소득은 각 거주자 소유 지분에 따라 과세됩니다."},
        {"output_answer": "이자 소득의 출처는 정부 및 지방자치단체가 발행한 채권, 법인이 발행한 채권, 국내외 은행 예금 등입니다."},
        {"output_answer": "배당소득은 국내외 법인으로부터 받는 배당금 및 배분금, 기타 법에서 정한 소득을 포함합니다."}
    ],
    metadata= [
        {"contexts": "제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다."},
        {"contexts": "제1조의2(정의) “거주자”란 국내에 주소를 두거나 183일 이상의 거소를 둔 개인을 말한다."},
        {"contexts": "제1조의2(정의) “비거주자”란 거주자가 아닌 개인을 말한다."},
        {"contexts": "제1조의2(정의) “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다."},
        {"contexts": "제2조(납세의무) 거주자 및 국내원천소득이 있는 비거주자는 소득세를 납부할 의무가 있다."},
        {"contexts": "제3조(과세소득의 범위) 거주자는 법에서 규정한 모든 소득에 대해 과세되며, 비거주자는 국내원천소득에 대해서만 과세된다."},
        {"contexts": "제4조(소득의 구분) 소득은 종합소득, 퇴직소득, 양도소득으로 분류된다."},
        {"contexts": "제4조(소득의 구분) 종합소득은 이자소득, 배당소득, 사업소득, 근로소득, 연금소득 및 기타소득을 포함한다."},
        {"contexts": "제12조(비과세소득) 비과세 소득에는 공익신탁의 이익, 특정 사업소득 및 기타 법에서 정한 특정 소득이 포함된다."},
        {"contexts": "제5조(과세기간) 소득세의 과세기간은 매년 1월 1일부터 12월 31일까지이다."},
        {"contexts": "제6조(납세지) 거주자의 소득세 납세지는 주소지이며, 주소지가 없으면 거소지이다."},
        {"contexts": "제6조(납세지) 비거주자의 소득세 납세지는 국내사업장의 소재지이다. 국내사업장이 여러 곳인 경우 주된 사업장의 소재지이다."},
        {"contexts": "제6조(납세지) 납세지가 불분명한 경우에는 대통령령으로 정한다."},
        {"contexts": "제7조(원천징수 등의 경우의 납세지) 원천징수세액의 납세지는 원천징수자의 종류와 위치에 따라 결정된다."},
        {"contexts": "제8조(상속 등의 경우의 납세지) 납세자의 사망 시 상속인 또는 납세관리인의 주소지나 거소지가 납세지가 된다."},
        {"contexts": "제2조의3(신탁재산 귀속 소득에 대한 납세의무의 범위) 신탁 소득에 대한 납세의 범위는 신탁의 수익자가 해당 소득에 대해 납세의무를 진다."},
        {"contexts": "제14조(과세표준의 계산) 이자소득, 배당소득 및 기타 법에서 정한 소득은 원천징수 대상이다."},
        {"contexts": "제14조(과세표준의 계산) 공동 소유 자산의 양도소득은 각 거주자 소유 지분에 따라 과세된다."},
        {"contexts": "제16조(이자소득) 이자 소득의 출처는 정부 및 지방자치단체가 발행한 채권, 법인이 발행한 채권, 국내외 은행 예금 등이다."},
        {"contexts": "제17조(배당소득) 배당소득은 국내외 법인으로부터 받는 배당금 및 배분금, 기타 법에서 정한 소득을 포함한다."}
    ],
    dataset_id=dataset.id,
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

### 2. 평가 대상 (Target) 함수 정의

In [3]:
from llm import get_ai_response_for_eval

def predict_rag_answer(example: dict):
    """
    평가 대상 함수 (Target Function)
    - 데이터셋의 각 질문에 대해 챗봇이 답변을 생성
    - 평가를 위한 single-turn 답변 생성 함수 
    
    Args:
        example: 데이터셋의 한 예제 {"input_question": "질문..."}
    
    Returns:
        {"answer": "챗봇의 답변..."}
    """
    question = example["input_question"]
    response = get_ai_response_for_eval(question)
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [4]:
#### ✅ 1단계: Target 함수 테스트

print("=" * 60)
print("🧪 TEST 1: predict_rag_answer 함수 테스트")
print("=" * 60)

# 테스트 예제
test_example = {"input_question": "거주자는 누구인가요?"}

try:
    print(f"\n질문: {test_example['input_question']}")
    print("\n답변 생성 중...")
    
    result = predict_rag_answer(test_example)
    
    print("\n✅ 성공!")
    print(f"\n답변:\n{result['answer']}")
    print(f"\n검색된 문서 수: {len(result['contexts'])}")
    print(f"\n첫 번째 문서 미리보기:\n{result['contexts'][:200]}...")
    
    # 결과 구조 확인
    assert "answer" in result, "❌ 'answer' 키가 없습니다!"
    assert "contexts" in result, "❌ 'contexts' 키가 없습니다!"
    assert isinstance(result["answer"], str), "❌ answer가 문자열이 아닙니다!"
    assert isinstance(result["contexts"], str), "❌ contexts가 문자열이 아닙니다!"
    
    print("\n✅ 모든 검증 통과!")
    
except Exception as e:
    print(f"\n❌ 에러 발생: {type(e).__name__}")
    print(f"에러 메시지: {str(e)}")
    print("\n⚠️ 이 문제를 해결하고 다음으로 진행하세요!")

🧪 TEST 1: predict_rag_answer 함수 테스트

질문: 거주자는 누구인가요?

답변 생성 중...

✅ 성공!

답변:
소득세법 제6조(거주자의 개념)에 따르면, 거주자는 다음과 같이 정의됩니다. 

1. 대한민국에 주소를 두고 있는 개인
2. 대한민국에 183일 이상 거소를 둔 개인

따라서 위의 조건 중 하나라도 충족하는 개인은 거주자로 간주됩니다.

검색된 문서 수: 4202

첫 번째 문서 미리보기:
3. 「자본시장과 금융투자업에 관한 법률」 제251조제1항에 따른 집합투자업겸영보험회사의 특별계정

③ 비거주자의 소득은 제119조에 따라 구분한다.

[전문개정 2009. 12. 31.]



제4조(소득의 구분) ① 거주자의 소득은 다음 각 호와 같이 구분한다. <개정 2013. 1. 1., 2020. 12. 29.>

1. 종합소득

　이 법에 따라 ...

✅ 모든 검증 통과!


### 3. 평가자 (Evaluator) 정의
- 소득세 챗봇을 위한 5개의 평가자
    1. **Hallucination** (환각 방지) - Hub 프롬프트 rag-answer-hallucination
    2. **Correctness** (정확성) - Hub 프롬프트 rag-answer-vs-reference
    3. **Citation** (법령 인용) - 커스텀
    4. **Helpfulness** (완전성) - Hub 프롬프트 rag-answer-helpfulness

- Hub 프롬프트 사용방법: https://smith.langchain.com/hub


In [18]:
# 필요한 라이브러리 import
from langchain_openai import ChatOpenAI
import re

In [37]:
def hallucination_evaluator(run, example) -> dict:
    """
    환각 방지 평가: 답변이 검색된 문서에 근거하는가?
    - RAG 시스템의 핵심 품질 지표
    - 법률 챗봇에서 특히 중요 (없는 법을 만들어내면 안 됨)
    """
    # Hub에서 검증된 프롬프트 가져오기
    hallucinations_prompt = client.pull_prompt("rlm/rag-answer-hallucination")
    
    # LLM Judge (GPT-4o 사용, temperature=0으로 일관성 확보)
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    # 평가 실행
    score = (hallucinations_prompt | llm).invoke({
        "input": {
            "documents": run.outputs["contexts"]  # Facts로 전달
        },
        "output": run.outputs["answer"]  # LLM generation으로 전달
    })
    
    return {
        "key": "hallucination",
        "score": score["Score"]
    }

In [38]:
def correctness_evaluator(run, example) -> dict:
    """
    정확성 평가: 답변이 정답과 의미적으로 얼마나 일치하는가?
    - 단순 문자열 비교가 아닌 의미적 유사성 평가
    - LLM Judge를 활용하여 맥락을 이해하며 평가
    """
    # Hub에서 검증된 프롬프트 가져오기
    reference_prompt = client.pull_prompt("rlm/rag-answer-vs-reference")
    # LLM Judge
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    # 평가 실행
    score = (reference_prompt | llm).invoke({
        "question": example.inputs["input_question"],  # 질문
        "correct_answer": example.outputs["output_answer"],  # 정답
        "student_answer": run.outputs["answer"]  # 챗봇의 답변
    })
    
    return {
        "key": "correctness",
        "score": score["Score"]
    }

In [39]:
def citation_evaluator(run, example) -> dict:
    """
    법령 인용 평가: "제X조" 형식으로 출처를 명시했는가?
    - 법률 답변에서 가장 중요한 지표
    - 정규표현식으로 빠르게 체크 (LLM 호출 불필요)
    """
    answer = run.outputs["answer"]
    
    # "제X조" 패턴 찾기 (예: 제1조, 제12조, 제1조의2 등)
    pattern = r'제\s*\d+(?:조|條)(?:의\d+)?'
    citations = re.findall(pattern, answer)
    
    # 평가 로직
    if len(citations) >= 1:
        score = 1.0
        comment = f"법령 인용 {len(citations)}회 발견"
    else:
        score = 0.0
        comment = "법령 인용 없음"
    
    return {
        "key": "citation",
        "score": score,
        "comment": comment
    }

In [42]:
def helpfulness_evaluator(run, example) -> dict:
    """
    완전성 평가: 질문에 충분히 답변했는가?
    - 사용자 관점에서의 답변 품질 평가
    - 정답과 비교하지 않고 답변 자체의 가치를 평가
    """
    # Hub에서 검증된 프롬프트 가져오기
    helpfulness_prompt = client.pull_prompt("rlm/rag-answer-helpfulness")
    
    # LLM Judge
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    # 평가 실행
    score = (helpfulness_prompt | llm).invoke({
        "input": {
            "question": example.inputs["input_question"]  # 질문
        },
        "output": run.outputs["answer"]  # LLM generation으로 전달
    })
    
    return {
        "key": "helpfulness",
        "score": score["Score"]
    }

In [43]:
#### ✅ 2단계: Evaluator 개별 테스트

print("\n" + "=" * 60)
print("🧪 TEST 2: 각 Evaluator 테스트")
print("=" * 60)

# Mock 데이터 생성 (실제 평가 형식과 동일)
class MockRun:
    def __init__(self, answer, contexts):
        self.outputs = {"answer": answer, "contexts": contexts}

class MockExample:
    def __init__(self, input_question, output_answer):
        self.inputs = {"input_question": input_question}
        self.outputs = {"output_answer": output_answer}

# 테스트 데이터
test_answer = "소득세법 제1조의2에 따르면, 거주자는 국내에 주소를 두거나 183일 이상의 거소를 둔 개인을 말합니다."
test_contexts = "제1조의2(정의) 거주자란 국내에 주소를 두거나 183일 이상의 거소를 둔 개인을 말한다."
test_question = "거주자는 누구인가요?"
test_reference = "거주자는 한국에 주소를 두거나 183일 이상 거소를 둔 개인을 의미합니다."

mock_run = MockRun(test_answer, test_contexts)
mock_example = MockExample(test_question, test_reference)

# ========================================
# 1. 커스텀 Evaluator 테스트 (빠름, 무료)
# ========================================
print("\n📋 커스텀 Evaluator 테스트 (빠름, 비용 없음):")

custom_evaluators = [
    ("Citation", citation_evaluator),
]

for name, evaluator in custom_evaluators:
    try:
        print(f"  - {name}...", end=" ", flush=True)
        result = evaluator(mock_run, mock_example)
        print(f"✅ Score: {result['score']}")
        if 'comment' in result:
            print(f"    💬 {result['comment']}")
    except Exception as e:
        print(f"❌ 실패: {str(e)}")
        import traceback
        traceback.print_exc()

# ========================================
# 2. LLM Judge Evaluator 테스트 (느림, 유료)
# ========================================
print("\n🤖 LLM Judge Evaluator 테스트:")
print("   ⏱️  약 10-15초 소요")
print("   💰 약 $0.01 비용 (gpt-4o 3회 호출)")
print()

llm_evaluators = [
    ("Hallucination (환각 방지)", hallucination_evaluator),
    ("Correctness (정확성)", correctness_evaluator),
    ("Helpfulness (완전성)", helpfulness_evaluator),
]

for name, evaluator in llm_evaluators:
    try:
        print(f"  - {name}...", end=" ", flush=True)
        result = evaluator(mock_run, mock_example)
        print(f"✅ Score: {result['score']:.2f}")
    except Exception as e:
        print(f"❌ 실패!")
        print(f"\n에러: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        print("\n⚠️ 이 evaluator를 수정하고 다시 테스트하세요!")
        break

print("\n" + "=" * 60)
print("✅ 모든 Evaluator 검증 완료!")
print("=" * 60)
print("\n💡 모두 통과했다면 이제 evaluate() 실행 가능!")


🧪 TEST 2: 각 Evaluator 테스트

📋 커스텀 Evaluator 테스트 (빠름, 비용 없음):
  - Citation... ✅ Score: 1.0
    💬 법령 인용 1회 발견

🤖 LLM Judge Evaluator 테스트:
   ⏱️  약 10-15초 소요
   💰 약 $0.01 비용 (gpt-4o 3회 호출)

  - Hallucination (환각 방지)... ✅ Score: 1.00
  - Correctness (정확성)... ✅ Score: 3.00
  - Helpfulness (완전성)... ✅ Score: 1.00

✅ 모든 Evaluator 검증 완료!

💡 모두 통과했다면 이제 evaluate() 실행 가능!


### 4. LangSmith로 평가 실행: evaluate()

- 데이터셋: `income_tax_dataset` (20개 질문)
- Target 함수: `predict_rag_answer()` 
- `evaluate()` 함수로 평가를 실행합니다.
    - 20개 질문 × 4개 evaluator = 80개 평가 실행
    - LLM Judge 3개 → 약 60회 LLM 호출

In [44]:
from langsmith.evaluation import evaluate

# Evaluators 리스트 정의
evaluators = [
    hallucination_evaluator,      # ⭐ 환각 방지 (Hub)
    correctness_evaluator,         # ⭐ 정확성 (Hub)
    citation_evaluator,            # ⭐ 법령 인용 (커스텀)
    helpfulness_evaluator,         # 완전성 (Hub, 선택)
]

dataset_name = "income_tax_dataset"

experiment_results = evaluate(
    predict_rag_answer,           # 평가 대상 함수 
    data=dataset_name,            # 데이터셋 이름
    evaluators=evaluators,        # 4개의 evaluator
    experiment_prefix="소득세-챗봇-v1",  # 실험 이름 (한글 가능)
    metadata={
        "version": "v1.0",
        "model": "gpt-4o-mini", # 챗봇에 사용한 모델 
        "features": "keyword_dict + few_shot + RAG",
        "description": "Single-turn evaluation"
    }
)

View the evaluation results for experiment: '소득세-챗봇-v1-71a30f6c' at:
https://smith.langchain.com/o/5f099df2-fb80-4f5f-8a65-c2f1e77a0438/datasets/c93c0749-572c-469d-b52c-f8c8f80330d8/compare?selectedSessions=01c1d984-8e94-4f2d-9f28-e93b40c4f8c6




0it [00:00, ?it/s]

### 5. 평가 결과 해석

LangSmith 웹 UI에서 확인할 수 있는 정보:

**1. 전체 점수 요약**
- Hallucination: X/20 (할루시네이션 있음: 0, 없음: 1)
- Correctness: X/20 (높을수록 좋음, 0~3)
- Citation: X/20 (인용안됨: 0, 인용됨: 1)
- Helpfulness: X/20 (도움안됨: 0, 도움됨: 1)

**2. 개별 질문 분석**
- 어떤 질문에서 실패했는지
- 각 evaluator별 점수
- 실제 답변 vs 정답 비교

**3. 개선 방향 도출**
- Hallucination 높으면: RAG 검색 품질 개선
- Correctness 낮으면: 프롬프트 또는 모델 개선
- Citation 낮으면: 프롬프트에 "법령 인용 필수" 추가
- Legal Terminology 낮으면: Few-shot 예시 보강